# ACN 정제 공정 Yield 분석

이 노트북은 ACN 정제 공정의 데이터를 분석하여 Input source 대비 output 양(Yield)에 영향을 끼치는 인자를 찾는 것을 목표로 합니다.

## 분석 목표
1. **Feature Selection**: Yield에 영향을 주는 주요 인자 식별
2. **다변량 패턴 분석**: 공정 변수들 간의 복합적 패턴 탐지

## 데이터 컬럼 설명
- **No**: fore run (AN-01에 남은 폐기 물질 재사용 횟수)
- **F/R Level**: AN-10 품질값 분석 시기 (product tank의 %)
- **LC**: 제품의 품질
- **Source**: 회사 source (0/1 binary)
- **AN-50_*nm**: 원료의 품질값
- **AN-01**: splitter, **AN-20**: condenser
- **나머지**: 열 관련 변수들


In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료")


In [ ]:
# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(data_path=None, df=None):
    """
    ACN 정제 공정 데이터 로드 및 전처리
    """
    if df is not None:
        data = df.copy()
    elif data_path:
        data = pd.read_csv(data_path)
    else:
        raise ValueError("데이터 경로 또는 DataFrame을 제공해야 합니다.")
    
    print("=" * 80)
    print("ACN 정제 공정 데이터 전처리")
    print("=" * 80)
    print(f"원본 데이터 크기: {data.shape}")
    
    # 1. 최종 F/R Level에서 분석한 데이터만 필터링
    if 'Final_FR' in data.columns:
        max_fr_level = data['Final_FR'].max()
        data = data[data['Final_FR'] == max_fr_level].copy()
        print(f"최종 F/R Level 필터링 후 데이터 크기: {data.shape}")
        print(f"최종 F/R Level: {max_fr_level}")
    
    # 2. 품질값 정규화 (spec 기준)
    quality_columns = ['AN-10_200nm', 'AN-10_225nm', 'AN-10_250nm', 
                      'AN-50_200nm', 'AN-50_225nm', 'AN-50_250nm']
    
    # 품질값 정규화 (음수: spec out, 양수: spec in)
    for col in quality_columns:
        if col in data.columns:
            # 0을 기준으로 정규화 (실제 spec 값에 따라 조정 필요)
            data[f'{col}_normalized'] = data[col] - 0  # 실제 spec 값으로 변경 필요
    
    # 3. 데이터 타입 설정
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    
    # 범주형 변수
    categorical_columns = ['Source', 'IsBubbled', 'IsBothChillerOn']
    for col in categorical_columns:
        if col in data.columns:
            data[col] = data[col].astype('category')
    
    # 수치형 변수
    numeric_columns = [col for col in data.columns if col not in categorical_columns + ['Date']]
    for col in numeric_columns:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    print(f"전처리 완료 후 데이터 크기: {data.shape}")
    return data

# 데이터 로드 (실제 데이터 경로로 변경하세요)
# df = load_and_preprocess_data(data_path='your_data.csv')
# 또는
# df = load_and_preprocess_data(df=your_dataframe)

print("데이터 전처리 함수 정의 완료")


In [ ]:
# Feature Selection 분석 함수
def feature_selection_analysis(df):
    """
    Feature Selection 분석
    """
    print("\n" + "=" * 80)
    print("Feature Selection 분석")
    print("=" * 80)
    
    if 'Yield' not in df.columns:
        print("Yield 컬럼이 없습니다.")
        return None
    
    # 수치형 변수만 선택 (Yield 제외)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Yield' in numeric_cols:
        numeric_cols.remove('Yield')
    
    # 결측치가 있는 컬럼 제외
    numeric_cols = [col for col in numeric_cols if not df[col].isnull().all()]
    
    X = df[numeric_cols].fillna(df[numeric_cols].median())
    y = df['Yield'].fillna(df['Yield'].median())
    
    print(f"분석 대상 특성 수: {len(numeric_cols)}")
    print(f"분석 대상 샘플 수: {len(X)}")
    
    # 1. 상관관계 기반 Feature Selection
    print("\n1. 상관관계 기반 Feature Selection")
    print("-" * 50)
    
    correlations = {}
    for col in numeric_cols:
        if col in X.columns:
            corr_pearson, p_value_pearson = pearsonr(X[col], y)
            corr_spearman, p_value_spearman = spearmanr(X[col], y)
            correlations[col] = {
                'pearson_corr': corr_pearson,
                'pearson_pvalue': p_value_pearson,
                'spearman_corr': corr_spearman,
                'spearman_pvalue': p_value_spearman,
                'abs_pearson': abs(corr_pearson),
                'abs_spearman': abs(corr_spearman)
            }
    
    # 상관관계 결과 정렬
    corr_df = pd.DataFrame(correlations).T
    corr_df = corr_df.sort_values('abs_pearson', ascending=False)
    
    print("Yield와의 상관관계 (상위 15개):")
    print(corr_df[['pearson_corr', 'pearson_pvalue', 'spearman_corr', 'spearman_pvalue']].head(15).round(4))
    
    return corr_df

print("Feature Selection 함수 정의 완료")


In [ ]:
# 다변량 패턴 분석 함수
def multivariate_pattern_analysis(df, top_features=None):
    """
    다변량 패턴 분석
    """
    print("\n" + "=" * 80)
    print("다변량 패턴 분석")
    print("=" * 80)
    
    if 'Yield' not in df.columns:
        print("Yield 컬럼이 없습니다.")
        return None
    
    # 수치형 변수만 선택 (Yield 제외)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Yield' in numeric_cols:
        numeric_cols.remove('Yield')
    
    # 결측치 처리
    X = df[numeric_cols].fillna(df[numeric_cols].median())
    y = df['Yield'].fillna(df['Yield'].median())
    
    # 상위 특성 선택 (기본값: 상위 10개)
    if top_features is None:
        correlations = []
        for col in numeric_cols:
            if col in X.columns:
                corr, _ = pearsonr(X[col], y)
                correlations.append((col, abs(corr)))
        
        correlations.sort(key=lambda x: x[1], reverse=True)
        top_features = [col for col, _ in correlations[:10]]
    
    print(f"분석 대상 특성: {top_features}")
    
    X_selected = X[top_features]
    
    # 데이터 표준화
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    
    # PCA 분석
    print("\n1. 주성분 분석 (PCA)")
    print("-" * 50)
    
    pca = PCA()
    pca_result = pca.fit_transform(X_scaled)
    
    # 설명 분산 비율
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    print("주성분별 설명 분산 비율:")
    for i, (var_ratio, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
        print(f"PC{i+1}: {var_ratio:.4f} (누적: {cum_var:.4f})")
    
    # 95% 분산을 설명하는 주성분 수
    n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"\n95% 분산을 설명하는 주성분 수: {n_components_95}")
    
    return {
        'pca': {
            'explained_variance_ratio': explained_variance_ratio,
            'cumulative_variance': cumulative_variance,
            'n_components_95': n_components_95
        }
    }

print("다변량 패턴 분석 함수 정의 완료")


In [ ]:
# 시각화 함수
def create_visualizations(df, feature_results=None, pattern_results=None):
    """
    시각화 생성
    """
    print("\n" + "=" * 80)
    print("시각화 생성")
    print("=" * 80)
    
    # 1. Yield 분포
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    if 'Yield' in df.columns:
        plt.hist(df['Yield'], bins=30, alpha=0.7, edgecolor='black')
        plt.axvline(df['Yield'].median(), color='red', linestyle='--', 
                   label=f'중간값: {df["Yield"].median():.3f}')
        plt.xlabel('Yield')
        plt.ylabel('빈도')
        plt.title('Yield 분포')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # 2. 상관관계 히트맵
    plt.subplot(2, 2, 2)
    if 'Yield' in df.columns:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_cols) > 1:
            # 상위 10개 특성만 선택
            if feature_results is not None:
                top_features = feature_results.head(10).index.tolist()
                if 'Yield' not in top_features:
                    top_features.append('Yield')
                
                corr_subset = df[top_features].corr()
                sns.heatmap(corr_subset, annot=True, cmap='coolwarm', center=0, fmt='.3f')
                plt.title('상위 특성 간 상관관계')
    
    # 3. PCA 설명 분산 비율
    plt.subplot(2, 2, 3)
    if pattern_results and 'pca' in pattern_results:
        pca_data = pattern_results['pca']
        plt.plot(range(1, len(pca_data['explained_variance_ratio']) + 1), 
                pca_data['explained_variance_ratio'], 'bo-', label='개별')
        plt.plot(range(1, len(pca_data['cumulative_variance']) + 1), 
                pca_data['cumulative_variance'], 'ro-', label='누적')
        plt.xlabel('주성분')
        plt.ylabel('설명 분산 비율')
        plt.title('PCA 설명 분산 비율')
        plt.legend()
        plt.grid(True)
    
    # 4. 주요 특성별 Yield 산점도
    plt.subplot(2, 2, 4)
    if feature_results is not None and 'Yield' in df.columns:
        top_feature = feature_results.index[0]  # 가장 상관관계가 높은 특성
        if top_feature in df.columns:
            plt.scatter(df[top_feature], df['Yield'], alpha=0.6)
            plt.xlabel(top_feature)
            plt.ylabel('Yield')
            plt.title(f'{top_feature} vs Yield')
            plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("시각화 함수 정의 완료")


## 분석 실행

아래 셀들을 순서대로 실행하여 ACN 정제 공정의 Yield 분석을 수행합니다.

### 1단계: 데이터 로드
실제 데이터 파일 경로를 입력하거나 DataFrame을 준비하세요.


In [ ]:
# 데이터 로드 (실제 데이터 경로로 변경하세요)
# 예시 1: CSV 파일에서 로드
# df = load_and_preprocess_data(data_path='your_data.csv')

# 예시 2: DataFrame에서 로드
# df = load_and_preprocess_data(df=your_dataframe)

# 예시 3: 샘플 데이터 생성 (테스트용)
np.random.seed(42)
sample_data = {
    'No': np.random.randint(1, 6, 100),
    'Date': pd.date_range('2023-01-01', periods=100, freq='D'),
    'F/R Level': np.random.uniform(50, 100, 100),
    'AN-10_200nm': np.random.normal(0.5, 0.1, 100),
    'AN-10_225nm': np.random.normal(0.3, 0.05, 100),
    'AN-10_250nm': np.random.normal(0.2, 0.03, 100),
    'LC': np.random.uniform(0.8, 1.2, 100),
    'Source': np.random.choice([0, 1], 100),
    'AN-50_200nm': np.random.normal(0.4, 0.08, 100),
    'AN-50_225nm': np.random.normal(0.25, 0.04, 100),
    'AN-50_250nm': np.random.normal(0.15, 0.02, 100),
    'IsBubbled': np.random.choice([0, 1], 100),
    'IsBothChillerOn': np.random.choice([0, 1], 100),
    'Input_source': np.random.uniform(100, 200, 100),
    'Steam_pressure': np.random.uniform(1, 3, 100),
    'AN-01_temp': np.random.uniform(80, 120, 100),
    'AN-01_pressure': np.random.uniform(0.5, 2, 100),
    'Heating_time_min': np.random.uniform(30, 120, 100),
    'Stable_time_min': np.random.uniform(10, 60, 100),
    'Product_time_min': np.random.uniform(60, 180, 100),
    'CWS_temp': np.random.uniform(20, 40, 100),
    'CWS_pressure': np.random.uniform(2, 5, 100),
    'CWR_temp': np.random.uniform(15, 35, 100),
    'CU-01_curr_temp': np.random.uniform(70, 110, 100),
    'Final_FR': np.random.uniform(80, 100, 100),
    'Product_Level': np.random.uniform(60, 100, 100),
    'AN-01_remain': np.random.uniform(10, 50, 100),
    'AN-20_temp': np.random.uniform(25, 45, 100),
    'Avg_temp': np.random.uniform(70, 100, 100),
    'Yield': np.random.uniform(0.7, 0.95, 100)  # 목표 변수
}

df = load_and_preprocess_data(df=pd.DataFrame(sample_data))


In [ ]:
# 2단계: Feature Selection 분석
feature_results = feature_selection_analysis(df)


In [ ]:
# 3단계: 다변량 패턴 분석
pattern_results = multivariate_pattern_analysis(df)


In [ ]:
# 4단계: 시각화
create_visualizations(df, feature_results, pattern_results)


In [ ]:
# 5단계: 결과 해석 및 권장사항
print("\n" + "=" * 80)
print("ACN 정제 공정 Yield 분석 결과 요약")
print("=" * 80)

if feature_results is not None:
    print("\n🔍 주요 영향 인자 (상위 5개):")
    top_5_features = feature_results.head(5)
    for i, (feature, row) in enumerate(top_5_features.iterrows(), 1):
        print(f"{i}. {feature}: 상관계수 {row['pearson_corr']:.4f} (p-value: {row['pearson_pvalue']:.4f})")

if pattern_results is not None and 'pca' in pattern_results:
    pca_data = pattern_results['pca']
    print(f"\n📊 PCA 분석 결과:")
    print(f"- 95% 분산을 설명하는 주성분 수: {pca_data['n_components_95']}")
    print(f"- 첫 번째 주성분 설명 분산: {pca_data['explained_variance_ratio'][0]:.4f}")

print(f"\n📈 Yield 통계:")
if 'Yield' in df.columns:
    yield_stats = df['Yield'].describe()
    print(f"- 평균: {yield_stats['mean']:.4f}")
    print(f"- 표준편차: {yield_stats['std']:.4f}")
    print(f"- 변동계수: {(yield_stats['std']/yield_stats['mean']*100):.2f}%")

print(f"\n💡 권장사항:")
print("1. 상위 영향 인자들의 모니터링 강화")
print("2. 고상관관계 특성들의 최적 범위 설정")
print("3. 정기적인 공정 변수 점검 및 조정")
print("4. 이상치 탐지 시스템 구축")
